In [1]:
# Imports go here

import os
import pdfkit
import base64
import PyPDF2
import json
import base64
import qrcode
import uuid
import fitz  # PyMuPDF

from PyPDF2 import PdfReader, PdfWriter
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

In [11]:
# Initializing stuff
path_to_wkhtmltopdf = r'./wkhtmltopdf/bin/wkhtmltopdf.exe'

config = pdfkit.configuration(wkhtmltopdf=path_to_wkhtmltopdf)

opt = {
    'margin-top': '2in',
    'margin-bottom': '1in',
    'margin-left': '1in',
    'margin-right': '1in'
}

temp_pdf_path = "./CacheData/temp.pdf"
final_pdf_path = "./CacheData/output.pdf"

In [12]:
# Separate them for url vs file loc
# Making the temp pdf
path_to_file = 'https://everyday-reading.com/blog-reading/'
pdfkit.from_url(path_to_file, output_path = temp_pdf_path, configuration = config, options=opt, verbose=True)


Loading pages (1/6)
Error: Failed to load https://z-na.amazon-adsystem.com/widgets/onejs?MarketPlace=US&adInstanceId=147edc6f-d05f-4840-8826-e90cff91b766, with network status code 3 and http status code 0 - Host  not found
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
Exit with code 1 due to network error: HostNotFoundError


True

In [13]:
# Count number of pages in pdf
def count_pdf_pages(temp_pdf_path):
    with open(temp_pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)
    return num_pages

num_pages = count_pdf_pages(temp_pdf_path)

In [14]:
# Generate uuid
uid = str(uuid.uuid4())

In [15]:
# Make folders to store QR
folder_path = "./QR"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    
uid_folder_path = os.path.join(folder_path, uid)
if not os.path.exists(uid_folder_path):
    os.makedirs(uid_folder_path)


In [16]:
# Make the QRs    
for p_no in range(num_pages):
    text = '''{{
            "id": "{}",
            "page": {}
        }}'''.format(uid, (p_no+1))
    # Create QR code instance
    qr = qrcode.QRCode(
        version = 1,
        error_correction = qrcode.constants.ERROR_CORRECT_L,
        box_size = 10,
        border = 4,
    )

    # Add data to QR code
    qr.add_data(text)
    qr.make(fit=True)
    
    # Create an image from the QR Code instance
    img = qr.make_image(fill='black', back_color='white')
    
    
    # Save the image
    ext = ".png"
    file_name = os.path.join(uid_folder_path, str(p_no) + ext)
    img.save(file_name)

In [17]:
# Making the pdf
pdf_reader = PdfReader(temp_pdf_path)
pdf_writer = PdfWriter()

In [19]:
for i in range(num_pages):
    page = pdf_reader.pages[i]

    # Fetch the corresponding QR code image and AR marker
    qr_filename = "{}.png".format(i)
    qr_path = os.path.join(uid_folder_path, qr_filename)
    ar_marker_path = '../../_ARMarker/Markers/MarkerIcons03.png'
    with open(ar_marker_path, 'rb') as marker_file:
        marker_data = marker_file.read()
        marker_base64 = base64.b64encode(marker_data).decode('utf-8')

    # Check if QR code file exists
    if os.path.exists(qr_path):
        # Convert the QR image to a base64-encoded string
        with open(qr_path, 'rb') as qr_file:
            qr_data = qr_file.read()
            qr_base64 = base64.b64encode(qr_data).decode('utf-8')
            
        # Combine QR code with AR marker
    # Create a PDF page from the PNG images
    image_pdf_path = 'image_page.pdf'
    c = canvas.Canvas(image_pdf_path, pagesize=letter)
    # Draw the AR marker
    c.drawImage("data:image/png;base64," + marker_base64, 205, 710, width=80, height=80)
    # Draw the QR code
    c.drawImage("data:image/png;base64," + qr_base64, 295, 700, width=100, height=100)
    c.save()

    # Merge the image page with the current page of the original PDF
    with open(image_pdf_path, 'rb') as image_pdf_file:
        image_pdf_reader = PdfReader(image_pdf_file)
        image_page = image_pdf_reader.pages[0]
        page.merge_page(image_page)
        pdf_writer.add_page(page)

    os.remove(image_pdf_path)  
    
with open(final_pdf_path, 'wb') as output_pdf:
    pdf_writer.write(output_pdf)

os.remove(temp_pdf_path)

In [20]:
import requests

URL = 'https://api.jsonbin.io/v3/b'

HEADERS = {
  'Content-Type': 'application/json',
  'X-Master-Key': '$2a$10$Z6B/b820H97jIfVer0qXVem.TJ5c69MZdAppCDKQ/1C6KwBtYYX/2'
}

# Function decorator to print status messages
def print_status(func):
    def wrapper(*args, **kwargs):
        print(f"Executing {func.__name__}...")
        result = func(*args, **kwargs)
        if result is not None:
            print("Success.")
        else:
            print("Failed.")
        return result
    return wrapper

# CRUD Operations
@print_status
def create_data():
    session = requests.Session()
    try:
        response = session.post(URL, json={"empty": "empty"}, headers=HEADERS)
        response.raise_for_status()  # Raise exception for non-200 status codes
        bin_id = response.json()['metadata']['id']
        return bin_id
    except requests.exceptions.RequestException as e:
        print(f"Error occurred: {e}")
        return None
    finally:
        session.close()

@print_status
def update_data(bin_id, json_data):
    session = requests.Session()
    try:
        cache_url = f"{URL}/{bin_id}"
        response = session.put(cache_url, json=json_data, headers=HEADERS)
        response.raise_for_status()  # Raise exception for non-200 status codes
        return True
    except requests.exceptions.RequestException as e:
        print(f"Error occurred: {e}")
        return False
    finally:
        session.close()

In [21]:
# Extract hyperlink info and create JSON 
def get_ar_marker_coordinates(pdf_path):
    # Open the PDF file
    pdf_document = fitz.open(pdf_path)
    _image_list = pdf_document.get_page_images(pno=0, full=True)
    ar_marker_coordinates = pdf_document[0].get_image_rects(_image_list[0][7], transform=True)[0][0]
    # Close the PDF document
    pdf_document.close()
    return ar_marker_coordinates



ar_marker_coordinates = get_ar_marker_coordinates(final_pdf_path)


# The code below here is for fetching hyperlink coordinates, and generating a json that looks like this:

# {
#     "ar_marker_coordinates": [<x0, <y0>, <x1>, <y1>],
#     "pages": [
#         {
#             "hyperlinks": [
#                 {
#                     "uri": "<url>", "coordinates": [<x0>, <y0>, <x1>, <y1>]
#                 }
#             ]
#         }
#     ]
# }

doc = fitz.open(final_pdf_path)

# Create dictionary representing json data we want to store on our server
json_data = {}
json_data['ar_marker_coordinates'] = [ar_marker_coordinates.x0, ar_marker_coordinates.y0,
                                      ar_marker_coordinates.x1, ar_marker_coordinates.y1]
json_data['pages'] = []

# Get the total page count
total_pages = doc.page_count
item_count = 0
hyperlink_id = create_data()

for page_idx in range(total_pages):
    cur_page = doc.load_page(page_idx)
    links = cur_page.get_links()

    # Convert the list of dictionaries to the required format
    hyperlinks = []

    for item in links:
        x0, y0, x1, y1 = item['from']
        coordinates = [round(coord, 5) for coord in [x0, y0, x1, y1]]
        uri = item.get('uri', '')
        hyperlink = {'id': hyperlink_id + "-" + str(item_count), 'uri': uri, 'coordinates': coordinates}
        hyperlinks.append(hyperlink)
        item_count += 1

    # Create the final dictionary
    json_data['pages'].append({"hyperlinks": hyperlinks})

update_data(bin_id=hyperlink_id, json_data=json_data)
# Print the result
print(json_data)

doc.close()

Executing create_data...
Success.
Executing update_data...
Success.
{'ar_marker_coordinates': [256.86151123046875, 180.01199340820312, 338.4886779785156, 258.0379638671875], 'pages': [{'hyperlinks': [{'id': '6640d18aacd3cb34a846c78f-0', 'uri': 'https://everyday-reading.com/book-logging/', 'coordinates': [361.5, 148.5, 408.75, 156.0]}, {'id': '6640d18aacd3cb34a846c78f-1', 'uri': 'https://everyday-reading.com/', 'coordinates': [93.75, 180.0, 499.5, 258.0]}, {'id': '6640d18aacd3cb34a846c78f-2', 'uri': 'https://everyday-reading.com/about', 'coordinates': [72.0, 144.0, 72.0, 144.0]}]}, {'hyperlinks': [{'id': '6640d18aacd3cb34a846c78f-3', 'uri': '', 'coordinates': [144.75, 559.5, 252.75, 567.75]}, {'id': '6640d18aacd3cb34a846c78f-4', 'uri': '', 'coordinates': [78.0, 654.75, 99.75, 664.5]}, {'id': '6640d18aacd3cb34a846c78f-5', 'uri': '', 'coordinates': [144.75, 699.0, 252.75, 707.25]}, {'id': '6640d18aacd3cb34a846c78f-6', 'uri': 'https://www.facebook.com/sharer/sharer.php?u=https%253A%252F%25